In [1]:
path_to_cogwheel = '/home/isha/PopulationSelectionEffects/cogwheel'

import sys
sys.path.append(path_to_cogwheel)

import pandas as pd

from cogwheel import data
from cogwheel import gw_prior
from cogwheel import likelihood
from cogwheel import sampling
from cogwheel import waveform
from cogwheel.posterior import Posterior


import subprocess
from matplotlib import pyplot as plt
import gwosc
from cogwheel import data

In [2]:
eventnames = sorted(path.name.removesuffix('.npz')
                    for path in data.DATADIR.glob('*.npz'))
print('Available eventnames:')
len(eventnames)

Available eventnames:


86

In [3]:
waveform.APPROXIMANTS

{'IMRPhenomD_NRTidalv2': Approximant(harmonic_modes=[(2, 2)], aligned_spins=True, tides=True),
 'IMRPhenomD_NRTidal': Approximant(harmonic_modes=[(2, 2)], aligned_spins=True, tides=True),
 'SEOBNRv4_ROM_NRTidal': Approximant(harmonic_modes=[(2, 2)], aligned_spins=True, tides=True),
 'SEOBNRv4_ROM_NRTidalv2': Approximant(harmonic_modes=[(2, 2)], aligned_spins=True, tides=True),
 'TaylorF2NLTides': Approximant(harmonic_modes=[(2, 2)], aligned_spins=True, tides=True),
 'IMRPhenomD': Approximant(harmonic_modes=[(2, 2)], aligned_spins=True, tides=False),
 'IMRPhenomXPHM': Approximant(harmonic_modes=[(2, 2), (2, 1), (3, 3), (3, 2), (4, 4)], aligned_spins=False, tides=False),
 'IMRPhenomXAS': Approximant(harmonic_modes=[(2, 2)], aligned_spins=True, tides=False),
 'IMRPhenomXP': Approximant(harmonic_modes=[(2, 2)], aligned_spins=False, tides=False)}

In [4]:
for prior_name, prior_class in gw_prior.prior_registry.items():
    print(f'{prior_name}:\n    {prior_class.__doc__}\n'
          .replace('\n    \n', '\n'))

IASPrior:
    Precessing, flat in chieff, uniform luminosity volume.

AlignedSpinIASPrior:
    Aligned spin, flat in chieff, uniform luminosity volume.

TidalIASPrior:
    Aligned spin, flat in tidal parameters, flat in chieff, uniform luminosity volume

LVCPrior:
    Precessing, isotropic spins, uniform luminosity volume.

AlignedSpinLVCPrior:
    Aligned spin components from isotropic distribution, uniform
    luminosity volume.

IASPriorComovingVT:
    Precessing, flat in chieff, uniform comoving VT.

AlignedSpinIASPriorComovingVT:
    Aligned spin, flat in chieff, uniform comoving VT.

LVCPriorComovingVT:
    Precessing, isotropic spins, uniform comoving VT.

AlignedSpinLVCPriorComovingVT:
    Aligned spins from isotropic distribution, uniform comoving VT.

ExtrinsicParametersPrior:
    Uniform luminosity volume, fixed intrinsic parameters.

MarginalizedDistanceIASPrior:
    Prior for usage with ``MarginalizedDistanceLikelihood``.
    Similar to ``IASPrior`` except it does not incl

In [5]:
metadata = pd.read_csv(data.DATADIR/'events_metadata.csv', index_col=0)  # Chirp mass guesses

In [6]:
# Choose from the above options:
eventname = eventnames[0]
mchirp_guess = metadata['mchirp'][eventname]
approximant = 'IMRPhenomXPHM'
prior_name = 'IASPrior'

post = Posterior.from_event(eventname, mchirp_guess, approximant, prior_name)

Searching incoherent solution for GW150914
Set intrinsic parameters, lnL = 294.1985459035243
Set time, lnL(H) = 204.09501966595323
Set sky location, lnL = 290.55209012224213
Set phase and distance, lnL = 290.55209012224213
Set mchirp_range = (25.663617075515507, 41.98824075412525)


In [7]:
pym = sampling.PyMultiNest(post)

In [8]:
pym.run_kwargs['n_live_points'] = 512

In [9]:
parentdir = '/home/isha/PopulationSelectionEffects/cogwheel_population_inference/cogwheel/cogwheel/example'
sampler = pym  # or `dyn`
rundir = sampler.get_rundir(parentdir)

In [10]:
# Run "live" (will take a while):
sampler.submit_slurm(rundir)

Submitted batch job 379150
Submitted job 'run_2_GW150914_IASPrior_PyMultiNest'.
